In [1]:
# Import the libraries
import os
from PIL import Image
from PIL.ExifTags import TAGS
import pandas as pd
from fractions import Fraction

In [2]:
# Function to extract exif metadata
def extract_exif_data(image_path):
    image = Image.open(image_path)
    exifdata = image._getexif()

    # Initialize variables to store extracted data
    image_name = os.path.basename(image_path)
    latitude, longitude, altitude = None, None, None

    for tag_id, data in exifdata.items():
        tag = TAGS.get(tag_id, tag_id)
        if tag == 'GPSInfo':
            gps_info = data
            latitude = gps_info.get(2)  # Latitude in degrees, minutes, seconds format
            longitude = gps_info.get(4)  # Longitude in degrees, minutes, seconds format
            altitude = gps_info.get(6)  # Altitude in meters

    # Convert latitude and longitude to decimal degrees (rounded to 4 decimal places)
    if latitude and longitude:
        latitude_decimal = round(float(Fraction(latitude[0], 1)) + float(Fraction(latitude[1], 60)) + float(Fraction(latitude[2], 3600)), 4)
        longitude_decimal = round(float(Fraction(longitude[0], 1)) + float(Fraction(longitude[1], 60)) + float(Fraction(longitude[2], 3600)), 4)
    else:
        print(f"Warning: No GPS information found for {image_name}")

    return image_name, latitude_decimal, longitude_decimal, altitude

In [3]:
# Example usage:
image_folder_path = "./JPGs"
output_file_path = 'drone_data.dat'

# create an empty list to append the image metadata
data_list = []
for image_file in os.listdir(image_folder_path):
    if image_file.lower().endswith('.jpg'):
        image_path = os.path.join(image_folder_path, image_file)
        image_data = extract_exif_data(image_path)
        data_list.append(image_data)

# Create a DataFrame from the extracted data
df = pd.DataFrame(data_list, columns=['imageName', 'latitude', 'longitude', 'altitude'])

# Export to .dat file (rounded to 4 decimal places)
df.to_csv(output_file_path, sep=' ', index=False)

print(f"Data exported to {output_file_path}")


Data exported to drone_data.dat
